<a href="https://colab.research.google.com/github/cjakuc/DS-Unit2_Build-Week/blob/master/Data%20Cleaning%20and%20Modeling/Unit2BuildWeekFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read in and wrangle hourly CSVs

In [0]:
# Read in CSVs
df_energy = pd.read_csv('https://github.com/cjakuc/DS-Unit1-Build-Week/blob/master/Energy%20Data/pjm_dayton_clean.csv?raw=true',
                        parse_dates=True,
                        index_col=0)
df_daily_temp = pd.read_csv('https://github.com/cjakuc/DS-Unit1-Build-Week/raw/master/Weather%20Data/dayton_daily_temp.csv',
                             parse_dates=True,
                             index_col=0)
df_hourly_temp = pd.read_csv('https://github.com/cjakuc/DS-Unit1-Build-Week/blob/master/Weather%20Data/dayton_hourly_temp.csv?raw=true',
                             parse_dates=True,
                             index_col=0,
                             header=None,
                             names=['AirTemp'])

In [0]:
# Add year, month, date, timestamp, and season variable to df_hourly_temp
df_hourly_temp['year'] = df_hourly_temp.index.year
df_hourly_temp['month'] = df_hourly_temp.index.month
df_hourly_temp['date'] = df_hourly_temp.index.dayofyear
df_hourly_temp['timestamp'] = df_hourly_temp.index.time
df_hourly_temp['season'] = (df_hourly_temp.index.month%12 + 3)//3
df_hourly_temp['season'] = df_hourly_temp['season'].replace({1:'winter',
                                     2:'spring',
                                     3:'summer',
                                     4:'fall'})
# Add year, month, date, timestamp, and season variable to df_daily_temp
df_daily_temp['year'] = df_daily_temp.index.year
df_daily_temp['month'] = df_daily_temp.index.month
df_daily_temp['date'] = df_daily_temp.index.dayofyear
df_daily_temp['timestamp'] = df_daily_temp.index.time
df_daily_temp['season'] = (df_daily_temp.index.month%12 + 3)//3
df_daily_temp['season'] = df_daily_temp['season'].replace({1:'winter',
                                     2:'spring',
                                     3:'summer',
                                     4:'fall'})

In [0]:
# Create a df that has the average MW of every hour, in each season

# Group data first by year, then by month
g = df_energy.groupby(['season','timestamp'])

# For each group, calculate the average of only the MW column
season_energy_hourly_averages = g.aggregate({'MW':np.mean})

# Create a df that has the average temp of every hour, in each season

# Group data first by year, then by month
g = df_hourly_temp.groupby(['season','timestamp'])

# For each group, calculate the average of only the temperature column
season_temp_hourly_averages = g.aggregate({'AirTemp':np.mean})


# Make 'Mean Hourly MW vs Mean Hourly Air Temperature at Dayton International Airport (2005-15)' plot

In [0]:
# Create seasonal temp and write to CSVs
summer_temp = season_temp_hourly_averages[season_temp_hourly_averages.index.get_level_values('season')=='summer']
# summer_temp.to_csv('summer_temp.csv')
spring_temp = season_temp_hourly_averages[season_temp_hourly_averages.index.get_level_values('season')=='spring']
# spring_temp.to_csv('spring_temp.csv')
winter_temp = season_temp_hourly_averages[season_temp_hourly_averages.index.get_level_values('season')=='winter']
# winter_temp.to_csv('winter_temp.csv')
fall_temp = season_temp_hourly_averages[season_temp_hourly_averages.index.get_level_values('season')=='fall']
# fall_temp.to_csv('fall_temp.csv')
# Create seasonal energy and write to CSVs
summer_energy = season_energy_hourly_averages[season_energy_hourly_averages.index.get_level_values('season')=='summer']
# summer_energy.to_csv('summer_energy.csv')
spring_energy = season_energy_hourly_averages[season_energy_hourly_averages.index.get_level_values('season')=='spring']
# spring_energy.to_csv('spring_energy.csv')
winter_energy = season_energy_hourly_averages[season_energy_hourly_averages.index.get_level_values('season')=='winter']
# winter_energy.to_csv('winter_energy.csv')
fall_energy = season_energy_hourly_averages[season_energy_hourly_averages.index.get_level_values('season')=='fall']
# fall_energy.to_csv('fall_energy.csv')

# Create the figure w/ two side by side subplots
fig = make_subplots(rows=1, cols=2, shared_xaxes=False,
                    subplot_titles=('Mean Hourly MW','Mean Hourly Air Temperature'),
                    x_title='Timestamp')
# Create each individual line for each season of the energy demand
fig.add_trace(go.Scatter(x=summer_energy.index.get_level_values('timestamp'),
                         y=summer_energy['MW'],
                         line=dict(color='#2CA02D'),
                         name='Summer'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=spring_energy.index.get_level_values('timestamp'),
                         y=spring_energy['MW'],
                         line=dict(color='#FC7F0F'),
                         name='Spring'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=winter_energy.index.get_level_values('timestamp'),
                         y=winter_energy['MW'],
                         line=dict(color='#2077B4'),
                         name='Winter'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=fall_energy.index.get_level_values('timestamp'),
                         y=fall_energy['MW'],
                         line=dict(color='#D72829'),
                         name='Fall'),
              row=1,col=1)
# Create each individual line for each season of the air temperature
fig.add_trace(go.Scatter(x=summer_temp.index.get_level_values('timestamp'),
                         y=summer_temp['AirTemp'],
                         line=dict(color='#2CA02D'),
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=spring_temp.index.get_level_values('timestamp'),
                         y=spring_temp['AirTemp'],
                         line=dict(color='#FC7F0F'),
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=winter_temp.index.get_level_values('timestamp'),
                         y=winter_temp['AirTemp'],
                         line=dict(color='#2077B4'),
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=fall_temp.index.get_level_values('timestamp'),
                         y=fall_temp['AirTemp'],
                         line=dict(color='#D72829'),
                         showlegend=False),
              row=1,col=2)
# Add a title
fig.update_layout(
    title='Mean Hourly MW vs Mean Hourly Air Temperature at Dayton International Airport (2005-15)'
)
# Update x axis properties
fig.update_xaxes(tickvals=['00:00:00','02:00:00','04:00:00','06:00:00',
                           '08:00:00','10:00:00','12:00:00','14:00:00',
                           '16:00:00','18:00:00','20:00:00','22:00:00'],
                 ticktext=['00:00','02:00','04:00','06:00','08:00','10:00',
                           '12:00','14:00','16:00','18:00','20:00','22:00'],
                 tickangle=45)
# Update y axis properties
fig.update_yaxes(title_text='MW',
                 row=1,col=1)
fig.update_yaxes(title_text='Temperature (F)',
                 row=1,col=2)

### Create the figure from the zipped file of CSVs

In [0]:
# Practice unzipping the zipped CSVs
import requests, zipfile, io
r = requests.get('https://github.com/cjakuc/DS-Unit2_Build-Week/blob/master/Data/MeanHourly.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
fall_energy = pd.read_table(z.open('fall_energy.csv'),delimiter=',')
spring_energy = pd.read_table(z.open('spring_energy.csv'),delimiter=',')
summer_energy = pd.read_table(z.open('summer_energy.csv'),delimiter=',')
winter_energy = pd.read_table(z.open('winter_energy.csv'),delimiter=',')
fall_temp = pd.read_table(z.open('fall_temp.csv'),delimiter=',')
spring_temp = pd.read_table(z.open('spring_temp.csv'),delimiter=',')
summer_temp = pd.read_table(z.open('summer_temp.csv'),delimiter=',')
winter_temp = pd.read_table(z.open('winter_temp.csv'),delimiter=',')

In [0]:
# Create the figure w/ two side by side subplots
fig = make_subplots(rows=1, cols=2, shared_xaxes=False,
                    subplot_titles=('Mean Hourly MW','Mean Hourly Air Temperature'),
                    x_title='Timestamp')
# Create each individual line for each season of the energy demand
fig.add_trace(go.Scatter(x=summer_energy['timestamp'],
                         y=summer_energy['MW'],
                         line=dict(color='#2CA02D'),
                         name='Summer',
                         hovertext='(Time, MW)'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=spring_energy['timestamp'],
                         y=spring_energy['MW'],
                         line=dict(color='#FC7F0F'),
                         name='Spring',
                         hovertext='(Time, MW)'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=winter_energy['timestamp'],
                         y=winter_energy['MW'],
                         line=dict(color='#2077B4'),
                         name='Winter',
                         hovertext='(Time, MW)'),
              row=1,col=1)
fig.add_trace(go.Scatter(x=fall_energy['timestamp'],
                         y=fall_energy['MW'],
                         line=dict(color='#D72829'),
                         name='Fall',
                         hovertext='(Time, MW)'),
              row=1,col=1)
# Create each individual line for each season of the air temperature
fig.add_trace(go.Scatter(x=summer_temp['timestamp'],
                         y=summer_temp['AirTemp'],
                         line=dict(color='#2CA02D'),
                         showlegend=False,
                         name='Summer',
                         hovertext='(Time, Temperature)'),
              row=1,col=2)
fig.add_trace(go.Scatter(x=spring_temp['timestamp'],
                         y=spring_temp['AirTemp'],
                         line=dict(color='#FC7F0F'),
                         showlegend=False,
                         name='Spring',
                         hovertext='(Time, Temperature)'),
              row=1,col=2)
fig.add_trace(go.Scatter(x=winter_temp['timestamp'],
                         y=winter_temp['AirTemp'],
                         line=dict(color='#2077B4'),
                         showlegend=False,
                         name='Winter',
                         hovertext='(Time, Temperature)'),
              row=1,col=2)
fig.add_trace(go.Scatter(x=fall_temp['timestamp'],
                         y=fall_temp['AirTemp'],
                         line=dict(color='#D72829'),
                         showlegend=False,
                         name='Fall',
                         hovertext='(Time, Temperature)'),
              row=1,col=2)
# Add a title
title = {'xref':'paper', 'yref':'paper', 'x':0.0,
                              'xanchor':'left', 'yanchor':'bottom',
                              'text':'Mean Hourly MW vs Mean Hourly Air Temperature (2005-15)',
                              'font':dict(family='Arial',
                                        size=20)}
fig.update_layout(title=title,template='plotly_dark')
# Update x axis properties
fig.update_xaxes(tickvals=['00:00:00','02:00:00','04:00:00','06:00:00',
                           '08:00:00','10:00:00','12:00:00','14:00:00',
                           '16:00:00','18:00:00','20:00:00','22:00:00'],
                 ticktext=['00:00','02:00','04:00','06:00','08:00','10:00',
                           '12:00','14:00','16:00','18:00','20:00','22:00'],
                 tickangle=45)
# Update y axis properties
fig.update_yaxes(title_text='MW', patch=dict(title=dict(standoff=0)),
                 row=1,col=1)
fig.update_yaxes(title_text='Temperature (F)', patch=dict(title=dict(standoff=0)),
                 row=1,col=2)
# Update line widths
fig.update_traces(line=dict(width=3))
# Add footer
annotations = []
# Add footer
fig.add_annotation(dict(xref='paper', yref='paper', x=-0.1, y=-0.25,
                              xanchor='left', yanchor='bottom',
                              text='Source: data from DPL and NOAA',
                              font=dict(family='Arial',
                                        size=10,
                                        color='gray'),
                              showarrow=False))

# Make the 'Mean Total MW Consumed per Day (2005-2015)' plot

In [0]:
# Create a df that has the average MW of every day

# Group data first by year, then by month
g = df_energy.groupby(['date'])

# For each group, calculate the average of only the MW column
daily_energy_averages = g.aggregate({'MW':np.mean})
# daily_energy_averages.to_csv('daily_energy.csv')

# Create an array of the last day of each season
season_end = [59,151,243,334]
season_x_labels=['January 1st',
                 'February 19th',
                 'April 10th',
                 'May 30th',
                 'July 19th',
                 'September 7th',
                 'October 27th',
                 'December 16th']
season_x_ticks = [1,50,100,150,200,250,300,350]

# Make the plot
fig1 = px.line(daily_energy_averages,
              x=daily_energy_averages.index.get_level_values('date'),
              y='MW',
              labels={'x':'Day'},
              template = 'plotly_dark')
# Change the color of the line
fig1.update_traces(line=dict(color='gold',width=2))
# Add vertical lines at season beginning/endings
season_list = [] # Create a list of dict objects to create each individual line
for season in season_end:
  season_list.append(dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= season, x1= season,
      line=dict(color='green',dash='dash'),
      name='Season Change'
      ))
season_tuple = tuple(season_list) # shapes= expects a tuple so convert list to tuple
# Add dotted lines
fig1.update_layout(shapes=season_tuple)
# Add annotations for title and holidays
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Mean Total MW Consumed per Day (2005-2015)',
                              font=dict(family='Arial',
                                        size=20),
                              showarrow=False))
# Add subtitle
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.00,
                              xanchor='left', yanchor='bottom',
                              text='Dashed lines show change of seasons',
                              font=dict(family='Arial',
                                        size=15,
                                        color='gray'),
                              showarrow=False))
# Add footer
annotations.append(dict(xref='paper', yref='paper', x=-0.1, y=-0.25,
                              xanchor='left', yanchor='bottom',
                              text='Source: data from DPL',
                              font=dict(family='Arial',
                                        size=10,
                                        color='gray'),
                              showarrow=False))
  # Holidays
annotations.append(dict(
    x=185, y=1900, text="4th of July", font=dict(size=12),
    xref="x",yref="y", showarrow=True, arrowhead=7, ax=10, ay=40,))
annotations.append(dict(
    x=359, y=1770, text="Christmas", font=dict(size=12),
    xref="x",yref="y", showarrow=True, arrowhead=7, ax=-77, ay=0))
annotations.append(dict(
    x=1, y=1835, text="New Years Day", font=dict(size=12),
    xref="x",yref="y", showarrow=True, arrowhead=7, ax=42, ay=40))
fig1.update_layout(annotations=annotations,showlegend=True)
# Update x axis
fig1.update_xaxes(
                 tickvals=season_x_ticks,
                 ticktext=season_x_labels,
                 tickangle=15)
# Update y axis properties
fig1.update_yaxes(title_text='MW', patch=dict(title=dict(standoff=0)))

# Create the interactive model building for the predictions page

## Linear regression models

In [0]:
# Import data
df_wrangled = pd.read_csv('https://github.com/cjakuc/DS-Unit2_Build-Week/blob/master/Data/BuildWeek2FinalData.csv?raw=true',
                 infer_datetime_format=True,
                 index_col=0)

### Linear regression model with all features

In [0]:
# Split train and val
def wrangle(X):
  """Wrangle train, validate, and test in the same way"""

  X = X.copy()

  # Add lagged variables
  X['DailyAvgAirTemp_lag1'] = X['DailyAvgAirTemp'].shift(fill_value=0)
  X['DailyHeatingDegreeDays_lag1'] = X['DailyHeatingDegreeDays'].shift(fill_value=0)
  X['DailyCoolingDegreeDays_lag1'] = X['DailyCoolingDegreeDays'].shift(fill_value=0)
  X['DailyHeatingDegreeDays_lag24'] = X['DailyHeatingDegreeDays'].shift(24,fill_value=0)
  X['DailyCoolingDegreeDays_lag24'] = X['DailyCoolingDegreeDays'].shift(24,fill_value=0)
    # These were un-realistic
  X['DailyHeatingDegreeDays_lag365'] = X['DailyHeatingDegreeDays'].shift(365,fill_value=0)
  X['DailyCoolingDegreeDays_lag365'] = X['DailyCoolingDegreeDays'].shift(365,fill_value=0)
    # These were mostly unhelpful and I'm replacing them w/ vars for degrees
    # over/under 65
  # Degrees over 65
  X['DegOver65'] = [(x - 65) if x > 65 else 0 for x in X['HourlyDryBulbTemperature']]
  # # Degrees under 65
  X['DegUnder65'] = [(65 - x) if x < 65 else 0 for x in X['HourlyDryBulbTemperature']]
  # These didn't help
    # I assume it's because the random forest model automatically takes into
    # account this relationship with just the hourly temperature variable
    # Could be helpful in a linear model though

  X['MW_lag1'] = X['MW'].shift(fill_value=0)
  X['MW_lag24'] = X['MW'].shift(24,fill_value=0)
  X['MW_lag365'] = X['MW'].shift(365,fill_value=0)

  # # Drop unattainable/non-helpful info
  # X = X.drop(columns=['DailyAvgAirTemp',
  #                     'DailyHeatingDegreeDays',
  #                     'DailyCoolingDegreeDays',
  #                     'month'])

  return X

df_wrangled1 = wrangle(df_wrangled)
train = df_wrangled1[df_wrangled1['year']<2014]
val = df_wrangled1[df_wrangled1['year']==2014]
test = df_wrangled1[df_wrangled1['year']==2015]

In [0]:
df_wrangled1.to_csv('BW2_wrangled.csv')

In [0]:
df_wrangled1.columns

Index(['HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyRelativeHumidity',
       'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyVisibility',
       'HourlyWindDirection', 'HourlyWindSpeed', 'year', 'month', 'date',
       'hour', 'season', 'MW', 'DailyAvgAirTemp', 'DailyCoolingDegreeDays',
       'DailyHeatingDegreeDays', 'DailyAvgAirTemp_lag1',
       'DailyHeatingDegreeDays_lag1', 'DailyCoolingDegreeDays_lag1',
       'DailyHeatingDegreeDays_lag24', 'DailyCoolingDegreeDays_lag24',
       'DailyHeatingDegreeDays_lag365', 'DailyCoolingDegreeDays_lag365',
       'DegOver65', 'DegUnder65', 'MW_lag1', 'MW_lag24', 'MW_lag365'],
      dtype='object')

In [0]:
!pip install category_encoders==2.*
!pip install pdpbox
!pip install eli5
!pip install shap

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 57.7MB 70kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.0-cp36-none-any.whl size=57690722 sha256=87e3f859d1bebaeda4986157ef69877b400a3108b09ca99ffa2fcd30e639529f
  Stored in directory: /root/.cache/pip/wheels/7d/08/51/63fd122b04a2c87d780464eeffb94867c75bd96a64d500a3fe
Successfully built pdpbox
     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 276kB 2.8MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394155 sha256=cfddc79f58b859291f9122d3356566e8271674faa3ad8f680675f5aafd25f312
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [0]:
# Build linear regression model
from sklearn.linear_model import LinearRegression
import category_encoders as ce
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import eli5
from eli5.sklearn import PermutationImportance

# Select target
target = 'MW'
# Drop target from features
features = train.drop(columns=['MW']).columns.tolist()

# Arrange X features matrix & y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
y_test = test[target]

# Model
encoder = ce.OneHotEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
model = LinearRegression()

model.fit(X_train_encoded, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.

Using TensorFlow backend.


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# Pickle the linear unrealistic model
from joblib import dump
dump(model, 'linear_unrealistic.joblib', compress=True)

['linear_unrealistic.joblib']

In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'linear_unrealistic_all_permuter.joblib', compress=True)

['linear_unrealistic_all_permuter.joblib']

In [0]:
print('Train MAE', mean_absolute_error(y_train,model.predict(X_train_encoded)))
print('Validation MAE', mean_absolute_error(y_val,model.predict(X_val_encoded)))

Train MAE 48.03939709357255
Validation MAE 43.76412662857954


In [0]:
import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    model, 
    scoring='neg_mean_absolute_error', 
    n_iter=5, 
    random_state=42
)
permuter.fit(X_val_encoded, y_val)
feature_names = X_val_encoded.columns.tolist()
permutation_importances = eli5.show_weights(
    permuter, 
    top=None, # No limit: show permutation importances for all features
    feature_names=feature_names # must be a list
)
permutation_importances

Weight,Feature
425.5645 ± 3.3495,DailyAvgAirTemp_lag1
344.1061 ± 5.3894,DailyHeatingDegreeDays_lag1
327.7508 ± 5.2197,MW_lag1
144.3476 ± 2.7030,HourlySeaLevelPressure
132.3534 ± 1.1585,HourlyStationPressure
57.7095 ± 0.4699,DailyCoolingDegreeDays_lag1
52.7535 ± 1.1710,DegUnder65
26.5549 ± 0.4873,MW_lag365
26.0191 ± 0.5263,MW_lag24
25.5699 ± 0.4683,HourlyDewPointTemperature


#### Linear regression features for each model

In [0]:
linear_unrealistic_all = X_train_encoded.columns.tolist()
linear_unrealistic_best = X_train_encoded.drop(columns=[
                          'season_1','season_4',
                          'DailyAvgAirTemp',
                          'HourlyPrecipitation','season_2',
                          'HourlyWindDirection',
                          'year'
                          ]).columns.tolist()
linear_realistic_all = X_train_encoded.drop(columns=[
                          'MW_lag1','MW_lag24'
                          ]).columns.tolist()
linear_realistic_best = X_train_encoded.drop(columns=[
                          'DailyAvgAirTemp_lag1',
                          'MW_lag1',
                          'DailyCoolingDegreeDays_lag1',
                          'DailyCoolingDegreeDays',
                          'MW_lag24',
                          'year','HourlyWindDirection',
                          'season_3',
                          'DailyHeatingDegreeDays_lag365'
                          ]).columns.tolist()

In [0]:
# Fit and pickle the model for unrealistic_best
model.fit(X_train_encoded[linear_unrealistic_best],y_train)
dump(model, 'linear_unrealistic_best.joblib', compress=True)

['linear_unrealistic_best.joblib']

In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'linear_unrealistic_best_permuter.joblib', compress=True)

['linear_unrealistic_best_permuter.joblib']

### Linear regression w/ realistic features to figure out 'best' features using permutation importances

In [0]:
# Build linear regression model

# Select target
target = 'MW'
# Drop target from features
features = train.drop(columns=[target]).columns.tolist()

# Arrange X features matrix & y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

# Model
encoder = ce.OneHotEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
model = LinearRegression()

model.fit(X_train_encoded[linear_realistic_all], y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 80

In [0]:
import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    model, 
    scoring='neg_mean_absolute_error', 
    n_iter=5, 
    random_state=42
)
permuter.fit(X_val_encoded[linear_realistic_all], y_val)
feature_names = X_val_encoded[linear_realistic_all].columns.tolist()
permutation_importances = eli5.show_weights(
    permuter, 
    top=None, # No limit: show permutation importances for all features
    feature_names=feature_names # must be a list
)
permutation_importances

Weight,Feature
308.4945 ± 3.3804,HourlySeaLevelPressure
262.9747 ± 3.5822,HourlyStationPressure
223.7807 ± 5.2394,DegUnder65
202.6854 ± 1.7582,HourlyDewPointTemperature
53.6617 ± 1.3298,hour
34.2046 ± 1.1304,HourlyRelativeHumidity
34.0595 ± 1.0325,DailyHeatingDegreeDays_lag1
23.2041 ± 2.0651,DegOver65
18.1043 ± 1.2725,DailyHeatingDegreeDays_lag24
14.4852 ± 1.1518,DailyHeatingDegreeDays


In [0]:
# Pickle the linear realistic model
dump(model, 'linear_realistic.joblib', compress=True)

['linear_realistic.joblib']

In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'linear_realistic_all_permuter.joblib', compress=True)

['linear_realistic_all_permuter.joblib']

In [0]:
# Fit and pickle the model for realistic_best
model.fit(X_train_encoded[linear_realistic_best],y_train)
dump(model, 'linear_realistic_best.joblib', compress=True)
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'linear_realistic_best_permuter.joblib', compress=True)

['linear_realistic_best_permuter.joblib']

## XGBoost models

### XGBoost model with all features

In [0]:
# Build XGBoost model

from xgboost import XGBRegressor

# Drop target from features
features = train.drop(columns=[target]).columns.tolist()

# Arrange X features matrix & y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
X_test = test[features]
y_val = val[target]
y_test = test[target]

# Encode
encoder = ce.OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Build model
model = XGBRegressor(n_estimators=1000,
                      max_depth=6,
                      random_state=42,
                      learning_rate=0.5,
                      n_jobs=-1,
                      verbosity=0)
# Fit model
eval_set = [(X_train_encoded, y_train), 
            (X_val_encoded, y_val)]

model.fit(X_train_encoded, y_train, 
          eval_set=eval_set, 
          eval_metric='mae', 
          early_stopping_rounds=50,
          verbose=False)

# Print scores
print(f'XGB best iterations: {model.best_iteration}')
print('Train MAE', mean_absolute_error(y_train,model.predict(X_train_encoded)))
print('Validation MAE', mean_absolute_error(y_val,model.predict(X_val_encoded)))
print('Test MAE', mean_absolute_error(y_test,model.predict(X_test_encoded)))
print(f'Train R^2 Score: {model.score(X_train_encoded,y_train)}')
print(f'Validation R^2 Score: {model.score(X_val_encoded,y_val)}')
print(f'Test R^2 Score: {model.score(X_test_encoded,y_test)}\n')

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



XGB best iterations: 353
Train MAE 10.514691040768502
Validation MAE 19.132292251935286
Test MAE 19.13353790810633
Train R^2 Score: 0.9987630304183492
Validation R^2 Score: 0.9953512528293531
Test R^2 Score: 0.9951656449159686



In [0]:
# Pickle the XGBoost unrealistic model
dump(model, 'XGBoost_unrealistic.joblib', compress=True)

['XGBoost_unrealistic.joblib']

In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'xgboost_unrealistic_all_permuter.joblib', compress=True)

['xgboost_unrealistic_all_permuter.joblib']

In [0]:
import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    model, 
    scoring='neg_mean_absolute_error', 
    n_iter=5, 
    random_state=42
)
permuter.fit(X_val_encoded, y_val)
feature_names = X_val_encoded.columns.tolist()
permutation_importances = eli5.show_weights(
    permuter, 
    top=None, # No limit: show permutation importances for all features
    feature_names=feature_names # must be a list
)
permutation_importances

Weight,Feature
391.5776 ± 6.7787,MW_lag1
65.8732 ± 0.5460,hour
17.1504 ± 0.3639,HourlyDryBulbTemperature
6.5777 ± 0.3543,date
4.2944 ± 0.3121,DailyHeatingDegreeDays_lag24
2.9036 ± 0.2323,HourlySeaLevelPressure
2.4455 ± 0.1853,HourlyStationPressure
2.3501 ± 0.1472,HourlyDewPointTemperature
2.3125 ± 0.1514,MW_lag24
1.8422 ± 0.1163,DailyAvgAirTemp


#### XGBoost features for each model

In [0]:
xgboost_unrealistic_all = X_train_encoded.columns.tolist()
xgboost_unrealistic_best= X_train_encoded.drop(columns=[
                                'year','DegUnder65',
                                'DailyHeatingDegreeDays',
                                'DegOver65',
                                'DailyCoolingDegreeDays',
                                'DailyCoolingDegreeDays_lag1'
                                ]).columns.tolist()
xgboost_realistic_all = X_train_encoded.drop(columns=[
                                'MW_lag1','MW_lag24'
                                ]).columns.tolist()
xgboost_realistic_best = X_train_encoded.drop(columns=[
                                'MW_lag1','DegUnder65',
                                'DailyHeatingDegreeDays',
                                'DailyCoolingDegreeDays',
                                'DailyCoolingDegreeDays_lag1',
                                'MW_lag24','year',
                                'DegOver65'
                                ]).columns.tolist()

In [0]:
# Fit and pickle the model for unrealistic_best
model = XGBRegressor(n_estimators=1000,
                      max_depth=6,
                      random_state=42,
                      learning_rate=0.5,
                      n_jobs=-1,
                      verbosity=0)
eval_set = [(X_train_encoded[xgboost_unrealistic_best], y_train), 
            (X_val_encoded[xgboost_unrealistic_best], y_val)]
model.fit(X_train_encoded[xgboost_unrealistic_best], y_train,
          eval_set=eval_set, 
          eval_metric='mae', 
          early_stopping_rounds=50,
          verbose=False)
dump(model, 'XGBoost_unrealistic_best.joblib', compress=True)
print('Train MAE', mean_absolute_error(y_train,model.predict(X_train_encoded[xgboost_unrealistic_best])))
print('Validation MAE', mean_absolute_error(y_val,model.predict(X_val_encoded[xgboost_unrealistic_best])))
print('Test MAE', mean_absolute_error(y_test,model.predict(X_test_encoded[xgboost_unrealistic_best])))
print(f'Train R^2 Score: {model.score(X_train_encoded[xgboost_unrealistic_best],y_train)}')
print(f'Validation R^2 Score: {model.score(X_val_encoded[xgboost_unrealistic_best],y_val)}')
print(f'Test R^2 Score: {model.score(X_test_encoded[xgboost_unrealistic_best],y_test)}\n')

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



Train MAE 12.792563401016771
Validation MAE 19.17395440367259
Test MAE 19.376859486996754
Train R^2 Score: 0.9981698032257371
Validation R^2 Score: 0.9953937703514372
Test R^2 Score: 0.9951616398978984



In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'xgboost_unrealistic_best_permuter.joblib', compress=True)

['xgboost_unrealistic_best_permuter.joblib']

### XGBoost w/ realistic features to figure out 'best' features using permutation importances

In [0]:
# Drop target from features
features = train.drop(columns=[target]).columns.tolist()

# Arrange X features matrix & y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
X_test = test[features]
y_val = val[target]
y_test = test[target]

# Encode
encoder = ce.OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Build model
model = XGBRegressor(n_estimators=1000,
                      max_depth=6,
                      random_state=42,
                      learning_rate=0.5,
                      n_jobs=-1,
                      verbosity=0)
# Fit model
eval_set = [(X_train_encoded[xgboost_realistic_all], y_train), 
            (X_val_encoded[xgboost_realistic_all], y_val)]

model.fit(X_train_encoded[xgboost_realistic_all], y_train, 
          eval_set=eval_set, 
          eval_metric='mae', 
          early_stopping_rounds=50,
          verbose=False)
print('Train MAE', mean_absolute_error(y_train,model.predict(X_train_encoded[xgboost_realistic_all])))
print('Validation MAE', mean_absolute_error(y_val,model.predict(X_val_encoded[xgboost_realistic_all])))
print('Test MAE', mean_absolute_error(y_test,model.predict(X_test_encoded[xgboost_realistic_all])))
print(f'Train R^2 Score: {model.score(X_train_encoded[xgboost_realistic_all],y_train)}')
print(f'Validation R^2 Score: {model.score(X_val_encoded[xgboost_realistic_all],y_val)}')
print(f'Test R^2 Score: {model.score(X_test_encoded[xgboost_realistic_all],y_test)}\n')

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



Train MAE 110.67898012839466
Validation MAE 141.92633021803206
Test MAE 141.41452111943548
Train R^2 Score: 0.8767744100482737
Validation R^2 Score: 0.7824978833708873
Test R^2 Score: 0.7816333557266608



In [0]:
# Pickle the XGBoost realistic model
dump(model, 'XGBoost_realistic.joblib', compress=True)

['XGBoost_realistic.joblib']

In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'xgboost_realistic_all_permuter.joblib', compress=True)

['xgboost_realistic_all_permuter.joblib']

In [0]:
import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    model, 
    scoring='neg_mean_absolute_error', 
    n_iter=5, 
    random_state=42
)
permuter.fit(X_val_encoded[xgboost_realistic_all], y_val)
feature_names = X_val_encoded[xgboost_realistic_all].columns.tolist()
permutation_importances = eli5.show_weights(
    permuter, 
    top=None, # No limit: show permutation importances for all features
    feature_names=feature_names # must be a list
)
permutation_importances

Weight,Feature
134.9084 ± 2.8937,hour
68.3493 ± 1.1633,HourlyDryBulbTemperature
54.2424 ± 1.0474,DailyAvgAirTemp
22.5303 ± 0.9135,DailyHeatingDegreeDays_lag24
18.3668 ± 1.0709,DailyCoolingDegreeDays_lag24
16.4035 ± 1.3866,date
12.1886 ± 1.4451,DailyAvgAirTemp_lag1
7.2948 ± 0.8827,MW_lag365
6.9656 ± 0.7227,season
6.9474 ± 1.0609,DailyCoolingDegreeDays_lag365


In [0]:
# Fit and pickle the model for realistic_best
model = XGBRegressor(n_estimators=1000,
                      max_depth=6,
                      random_state=42,
                      learning_rate=0.5,
                      n_jobs=-1,
                      verbosity=0)
eval_set = [(X_train_encoded[xgboost_realistic_best], y_train), 
            (X_val_encoded[xgboost_realistic_best], y_val)]
model.fit(X_train_encoded[xgboost_realistic_best], y_train,
          eval_set=eval_set, 
          eval_metric='mae', 
          early_stopping_rounds=50,
          verbose=False)
dump(model, 'XGBoost_realistic_best.joblib', compress=True)
print('Train MAE', mean_absolute_error(y_train,model.predict(X_train_encoded[xgboost_realistic_best])))
print('Validation MAE', mean_absolute_error(y_val,model.predict(X_val_encoded[xgboost_realistic_best])))
print('Test MAE', mean_absolute_error(y_test,model.predict(X_test_encoded[xgboost_realistic_best])))
print(f'Train R^2 Score: {model.score(X_train_encoded[xgboost_realistic_best],y_train)}')
print(f'Validation R^2 Score: {model.score(X_val_encoded[xgboost_realistic_best],y_val)}')
print(f'Test R^2 Score: {model.score(X_test_encoded[xgboost_realistic_best],y_test)}\n')

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



Train MAE 122.25810238621172
Validation MAE 135.8921292100323
Test MAE 135.91781285432106
Train R^2 Score: 0.8509429565469353
Validation R^2 Score: 0.7806247531541045
Test R^2 Score: 0.7738162600896321



In [0]:
# Pickle the permuter
permuter = PermutationImportance(
  model, 
  scoring='neg_mean_absolute_error', 
  n_iter=5, 
  random_state=42
)
# permuter.fit(X_val_encoded, y_val)
dump(permuter, 'xgboost_realistic_best_permuter.joblib', compress=True)

['xgboost_realistic_best_permuter.joblib']

In [0]:
import joblib
import sklearn
import category_encoders as ce
import xgboost
print(f'joblib=={joblib.__version__}')
print(f'scikit-learn=={sklearn.__version__}')
print(f'category_encoders=={ce.__version__}')
print(f'xgboost=={xgboost.__version__}')

joblib==0.14.1
scikit-learn==0.22.1
category_encoders==2.1.0
xgboost==0.90


In [0]:
fig = px.box(df_wrangled1,x='hour',y='MW',
           template='plotly_dark',
          #  facet_row='season',
           range_y=[min(df_wrangled1['MW']),max(df_wrangled1['MW'])],
           labels={'hour':'Hour'})
title = {'xref':'paper', 'yref':'paper', 'x':0.0,'xanchor':'left',
        'yanchor':'bottom',
        'text':'Relationship Between MW and Hour of the Day',
        'font':dict(family='Arial',
                    size=20)}
fig.update_layout(title=title)

In [0]:
fig = px.line(data_frame=df_wrangled1, x=df_wrangled1.index,y='MW',
              labels={'x':'Year'},
              template='plotly_dark')
# Add a title
title = {'xref':'paper', 'yref':'paper', 'x':0.0,'xanchor':'left',
        'yanchor':'bottom',
        'text':'Baseline MW Predictions (2015)',
        'font':dict(family='Arial',
                    size=20)}
fig.update_layout(title=title)
fig.add_annotation(dict(xref='paper', yref='paper', x=0, y=1,
              xanchor='left', yanchor='bottom',
              text='White line is baseline predictions (MW mean)',
              font=dict(family='Arial', size=15,color='gray'),
              showarrow=False))
# Add horizontal red line
h_line = []
h_line.append(dict(
    type= 'line',
    yref= 'y', y0= df_wrangled1['MW'].mean(), y1= df_wrangled1['MW'].mean(),
    xref= 'paper', x0= 0, x1= 1,
    name='Baseline Estimate'))
fig.update_layout(shapes=h_line,showlegend=True)
fig

In [0]:
# Baseline MAE
mean_mw = pd.DataFrame(data={'actual':y_test,'mean':test['MW'].mean()})
baseline_mae = mean_absolute_error(y_test,mean_mw['mean'])
baseline_mae

301.48257370072764

In [0]:
test['MW'].mean()

1990.2337186677348